<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

Defaulting to user installation because normal site-packages is not writeable


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/0e/6b/87fcddd34df9f53880fa1f0c23af7b6b96c935856473faf3914323588c40/torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata


  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)


  Obtaining dependency information for nvidia-cudnn-cu12==9.5.1.17 from https://files.pythonhosted.org/packages/2a/78/4535c9c7f859a64781e43c969a3a7e84c54634e319a996d43ef32ce46f83/nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)


  Obtaining dependency information for nvidia-cufft-cu12==11.3.0.4 from https://files.pythonhosted.org/packages/8f/16/73727675941ab8e6ffd86ca3a4b7b47065edcca7a997920b831f8147c99d/nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata


...

Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (158.2 MB)


Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (216.6 MB)


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device: '/home/jala6738/.local/lib/python3.11/site-packages/nvidia/__init__.py'



None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [ ]:
dataset_dict = load_dataset("csv", data_files="data/gutenberg/uniform_excerpts_2.csv")

In [ ]:
df = pd.read_csv("data/gutenberg/uniform_excerpts_2.csv")

In [ ]:
label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df["label"])
df.head()


In [ ]:
train_df, val_test_df = train_test_split(df, test_size=0.15, stratify=df["label_id"], random_state=104, shuffle=True)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, stratify=val_test_df["label_id"], random_state=104, shuffle=True)


In [ ]:
train_dataset = Dataset.from_pandas(train_df[["text", "label_id"]].rename(columns={"label_id": "labels"}))
eval_dataset = Dataset.from_pandas(val_df[["text", "label_id"]].rename(columns={"label_id": "labels"}))
test_dataset = Dataset.from_pandas(test_df[["text", "label_id"]].rename(columns={"label_id": "labels"}))

In [ ]:

model_path = "google-bert/bert-base-uncased"



def tokenize_dataset(model_name, train_df, val_df, test_df):
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    def tokenize_function(examples):
        return tokenizer(
            examples["text"], truncation=True, padding="max_length", max_length=256
        )

    def prepare(dataset):
        dataset = Dataset.from_pandas(dataset[["text", "label_id"]].rename(columns={"label_id": "labels"}))
        dataset = dataset.map(tokenize_function, batched=True)
        dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
        return dataset

    return tokenizer, prepare(train_df), prepare(val_df), prepare(test_df)

In [ ]:
#hyperparamaters
lr = 1e-4
batch_size = 16
num_epochs = 4

def train_model(model_path, label_encoder, train_dataset, eval_dataset, tokenizer):
    training_args = TrainingArguments(
        output_dir="bert-english-classifier_teacher",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy"
    )


    def compute_metrics(p: EvalPrediction):
        preds = np.argmax(p.predictions, axis=1)
        return {"accuracy": accuracy_score(p.label_ids, preds)}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics
    )


    trainer.train()
    return trainer

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = len(label_encoder.classes_))

In [ ]:
'''#freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

#unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True'''

In [ ]:
#define text preprocessing
'''def preprocess_function(examples):
    #return tokenized text with truncation and padding
    return tokenizer(examples["text"], truncation=True, padding=True)'''

#preprocess all datasets
'''train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)'''


In [ ]:
tokenizer, train_dataset, val_dataset, test_dataset = tokenize_dataset(model_path, train_df, val_df, test_df)


In [ ]:

'''#hyperparameters
lr = 1e-4
batch_size = 16
num_epochs = 4

training_args = TrainingArguments(
    output_dir = "bert-english-classifier_teacher",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = "epoch",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
)'''

In [ ]:
'''#TODO: When running train_dataset and eval_dataset, there is a tensor error where the trainer cant create a tensor. From what I gather, this might be
# issue pertaining to the label in the uniform_excerpts dataset which isn't a flat integer.

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()'''
trainer = train_model(
    model_path=model_path,
    label_encoder=label_encoder,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

In [ ]:
results = trainer.evaluate(test_dataset)
print("Test Results:", results)
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids
print(classification_report(true_labels, pred_labels, target_names=label_encoder.classes_))